In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df = pd.read_csv("../../data/processed/train.csv")
test_df = pd.read_csv("../../data/processed/test.csv")

display(train_df.describe())
display(train_df.info())
display(train_df.head())

display(test_df.describe())
display(test_df.info())
display(test_df.head())

In [ ]:
X_train = train_df.drop("Revenue", axis=1)
y_train = train_df["Revenue"]

X_test = test_df.drop("Revenue", axis=1)
y_test = test_df["Revenue"]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# pip install ucimlrepo pandas scikit-learn numpy joblib

import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    f1_score, precision_recall_curve,
    confusion_matrix, classification_report
)

RANDOM_STATE = 42
TEST_SIZE = 0.2
MODEL_PATH = "online_shoppers_lr_robust_onehot.joblib"


# 2) Define columns (UCI schema 기준)
numeric_cols = [
    "Administrative", "Administrative_Duration",
    "Informational", "Informational_Duration",
    "ProductRelated", "ProductRelated_Duration",
    "BounceRates", "ExitRates",
    "PageValues", "SpecialDay",
]
categorical_cols = [
    "Month", "OperatingSystems", "Browser", "Region",
    "TrafficType", "VisitorType", "Weekend",
]

# 4) Preprocess (RobustScaler + OneHot)
preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ],
    remainder="drop",
)

# 5) Model with class_weight for imbalance
clf = LogisticRegression(
    max_iter=5000,
    solver="saga",           # sparse one-hot에 적합
    class_weight="balanced", # 불균형 가중치
    n_jobs=-1,
)

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", clf),
])

# 6) Fit
pipeline.fit(X_train, y_train)

# 7) Predict / Evaluate
proba = pipeline.predict_proba(X_test)[:, 1]
pred_default = (proba >= 0.5).astype(int)

roc = roc_auc_score(y_test, proba)
prauc = average_precision_score(y_test, proba)  # PR-AUC
f1 = f1_score(y_test, pred_default)

print("=== Evaluation (threshold=0.5) ===")
print(f"ROC-AUC : {roc:.4f}")
print(f"PR-AUC  : {prauc:.4f}")
print(f"F1      : {f1:.4f}")
print("Confusion matrix:\n", confusion_matrix(y_test, pred_default))
print("\nClassification report:\n", classification_report(y_test, pred_default, digits=4))

# (선택) PR 커브 기반으로 F1 최대가 되는 임계값도 같이 출력
prec, rec, thr = precision_recall_curve(y_test, proba)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = np.nanargmax(f1s)
best_thr = thr[best_idx - 1] if best_idx > 0 else 0.5  # thr 길이가 하나 짧음
print(f"\nBest-F1 threshold (from PR curve): {best_thr:.4f}, F1={f1s[best_idx]:.4f}")

# 8) Extract/Save model (전처리+모델 통째로 저장)
joblib.dump(pipeline, MODEL_PATH)
print(f"\nSaved pipeline to: {MODEL_PATH}")

# 9) (검증) 로드 후 동일 예측 되는지 간단 체크
loaded = joblib.load(MODEL_PATH)
proba2 = loaded.predict_proba(X_test)[:, 1]
print("Reload check (max abs diff in proba):", float(np.max(np.abs(proba - proba2))))


In [ ]:
# pip install matplotlib seaborn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    RocCurveDisplay, PrecisionRecallDisplay,
    confusion_matrix
)
from sklearn.calibration import calibration_curve
from sklearn.inspection import permutation_importance

sns.set(style="whitegrid")

# ===== 전제: 이미 아래 변수들이 존재한다고 가정 =====
# pipeline  : 학습된 Pipeline (preprocess + model)
# X_test    : 테스트 피처 (DataFrame)
# y_test    : 테스트 라벨 (0/1)
# proba     : pipeline.predict_proba(X_test)[:, 1]
# pred_default : (proba >= 0.5).astype(int)

# 1) ROC Curve
plt.figure(figsize=(6, 5))
RocCurveDisplay.from_predictions(y_test, proba)
plt.title("ROC Curve")
plt.tight_layout()
plt.show()

# 2) Precision-Recall Curve (불균형에서 중요)
plt.figure(figsize=(6, 5))
PrecisionRecallDisplay.from_predictions(y_test, proba)
plt.title("Precision-Recall Curve")
plt.tight_layout()
plt.show()

# 3) Confusion Matrix (threshold=0.5)
cm = confusion_matrix(y_test, pred_default)
plt.figure(figsize=(5.5, 4.5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (threshold=0.5)")
plt.tight_layout()
plt.show()

# 4) Calibration Curve (확률이 얼마나 믿을만한지)
prob_true, prob_pred = calibration_curve(y_test, proba, n_bins=10, strategy="quantile")
plt.figure(figsize=(6, 5))
plt.plot(prob_pred, prob_true, marker="o", label="Model")
plt.plot([0, 1], [0, 1], "--", label="Perfectly calibrated")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration Curve")
plt.legend()
plt.tight_layout()
plt.show()

# 5) Predicted probability distribution (구매/비구매 확률 분포가 갈라지는지)
plt.figure(figsize=(7, 4.5))
sns.kdeplot(proba[y_test == 0], label="Non-purchase (y=0)", fill=True, alpha=0.35)
sns.kdeplot(proba[y_test == 1], label="Purchase (y=1)", fill=True, alpha=0.35)
plt.xlabel("Predicted probability")
plt.title("Predicted Probability Distribution")
plt.legend()
plt.tight_layout()
plt.show()

# 6) Permutation Importance (Pipeline에서도 가능)
#    - OneHot까지 포함한 전체 피처 중요도는 컬럼이 너무 많아질 수 있어서
#      여기서는 “원본 컬럼 단위”로 보고 싶다면 별도 설계가 필요함.
#    - 대신: transformed feature 기준 top-k를 뽑아 보여줌.

# (주의) Pipeline에서 preprocess 결과는 sparse일 수 있음. permutation_importance는 동작하지만 느릴 수 있어.
result = permutation_importance(
    pipeline, X_test, y_test,
    n_repeats=10, random_state=42,
    scoring="average_precision"  # PR-AUC 기반 중요도
)

importances = result.importances_mean
idx = np.argsort(importances)[::-1]

# transformed feature names 얻기
pre = pipeline.named_steps["preprocess"]
feature_names = pre.get_feature_names_out()

top_k = 25
top_idx = idx[:top_k]

plt.figure(figsize=(9, 6))
sns.barplot(x=importances[top_idx], y=np.array(feature_names)[top_idx], orient="h")
plt.title(f"Permutation Importance (Top {top_k}, scoring=PR-AUC)")
plt.xlabel("Mean importance decrease")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


In [ ]:
# pip install pandas numpy scikit-learn joblib matplotlib seaborn

import numpy as np
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    f1_score, roc_auc_score, average_precision_score,
    precision_recall_curve, confusion_matrix, classification_report
)

# =========================
# Config
# =========================
TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"   # 네 csv에서 타깃 컬럼명
MODEL_PATH = "model_stacking_robust_onehot.joblib"
RANDOM_STATE = 42
N_SPLITS = 5

# =========================
# Load
# =========================
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

if TARGET_COL not in train_df.columns:
    raise ValueError(f"TARGET_COL='{TARGET_COL}' not found in train.csv columns: {list(train_df.columns)}")

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)  # True/False or 0/1 모두 대응(대부분 int로 변환 가능)

# test에도 라벨이 있다면 평가 가능, 없다면 예측만 가능하게 처리
has_test_label = TARGET_COL in test_df.columns
X_test = test_df.drop(columns=[TARGET_COL]) if has_test_label else test_df.copy()
y_test = test_df[TARGET_COL].astype(int) if has_test_label else None

# =========================
# Column typing (자동 추정)
# =========================
# - 숫자형: number dtype
# - 범주형: object/bool/category
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

# (안전) Month가 문자열인데 숫자로 읽혔거나, 반대일 수도 있음 → 필요하면 여기서 수동 지정
# print("num_cols:", num_cols)
# print("cat_cols:", cat_cols)

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

# =========================
# Models (불균형 weight 적용)
# =========================
lr = LogisticRegression(
    max_iter=5000, solver="saga", class_weight="balanced",
    n_jobs=-1, random_state=RANDOM_STATE
)

rf = RandomForestClassifier(
    n_estimators=400,
    min_samples_leaf=2,
    class_weight="balanced_subsample",
    n_jobs=-1,
    random_state=RANDOM_STATE
)

et = ExtraTreesClassifier(
    n_estimators=600,
    min_samples_leaf=2,
    class_weight="balanced",
    n_jobs=-1,
    random_state=RANDOM_STATE
)

meta = LogisticRegression(
    max_iter=5000, solver="saga", class_weight="balanced",
    n_jobs=-1, random_state=RANDOM_STATE
)

stack = StackingClassifier(
    estimators=[("lr", lr), ("rf", rf), ("et", et)],
    final_estimator=meta,
    stack_method="predict_proba",
    cv=5,
    n_jobs=-1
)

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", stack),
])

# =========================
# 1) OOF 확률로 threshold 튜닝 (train만 사용, 추가 split 없음)
# =========================
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
oof_proba = np.zeros(len(X_train), dtype=float)

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_train, y_train), 1):
    X_tr, y_tr = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    X_va, y_va = X_train.iloc[va_idx], y_train.iloc[va_idx]

    pipeline.fit(X_tr, y_tr)
    oof_proba[va_idx] = pipeline.predict_proba(X_va)[:, 1]
    print(f"[Fold {fold}] done")

prec, rec, thr = precision_recall_curve(y_train, oof_proba)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = np.nanargmax(f1s)
best_thr = thr[best_idx - 1] if best_idx > 0 else 0.5
print("\n[OOF] Best threshold:", float(best_thr), "Best F1:", float(f1s[best_idx]))

# =========================
# 2) train 전체로 재학습
# =========================
pipeline.fit(X_train, y_train)

# =========================
# 3) test 평가(라벨 있을 때)
# =========================
test_proba = pipeline.predict_proba(X_test)[:, 1]
test_pred = (test_proba >= best_thr).astype(int)

if has_test_label:
    print("\n=== TEST Metrics (using OOF-tuned threshold) ===")
    print("ROC-AUC :", roc_auc_score(y_test, test_proba))
    print("PR-AUC  :", average_precision_score(y_test, test_proba))
    print("F1      :", f1_score(y_test, test_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, test_pred))
    print("\nReport:\n", classification_report(y_test, test_pred, digits=4))
else:
    print("\n(test.csv에 Revenue가 없어서 평가 대신 예측만 생성합니다.)")

# =========================
# 4) 모델 추출/저장 (pipeline + threshold 같이 저장)
# =========================
artifact = {
    "pipeline": pipeline,
    "best_threshold": float(best_thr),
    "num_cols": num_cols,
    "cat_cols": cat_cols
}
joblib.dump(artifact, MODEL_PATH)
print("\nSaved:", MODEL_PATH)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay, confusion_matrix
from sklearn.calibration import calibration_curve

sns.set(style="whitegrid")

# 전제: y_test, test_proba, test_pred 가 존재해야 함 (test에 Revenue 있을 때)
plt.figure(figsize=(6,5))
RocCurveDisplay.from_predictions(y_test, test_proba)
plt.title("ROC Curve (TEST)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,5))
PrecisionRecallDisplay.from_predictions(y_test, test_proba)
plt.title("Precision-Recall Curve (TEST)")
plt.tight_layout()
plt.show()

cm = confusion_matrix(y_test, test_pred)
plt.figure(figsize=(5.5,4.5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title(f"Confusion Matrix (TEST, thr={artifact['best_threshold']:.3f})")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# Calibration curve (확률 신뢰도)
prob_true, prob_pred = calibration_curve(y_test, test_proba, n_bins=10, strategy="quantile")
plt.figure(figsize=(6,5))
plt.plot(prob_pred, prob_true, marker="o", label="Model")
plt.plot([0,1],[0,1],"--", label="Perfect")
plt.title("Calibration Curve (TEST)")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.legend()
plt.tight_layout()
plt.show()

# 확률 분포 (구매/비구매 분리 정도)
plt.figure(figsize=(7,4.5))
sns.kdeplot(test_proba[y_test==0], label="y=0", fill=True, alpha=0.35)
sns.kdeplot(test_proba[y_test==1], label="y=1", fill=True, alpha=0.35)
plt.title("Predicted Probability Distribution (TEST)")
plt.xlabel("Predicted probability")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# pip install pandas numpy scikit-learn imbalanced-learn joblib

import numpy as np
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_recall_curve, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report
)

from imblearn.ensemble import BalancedRandomForestClassifier

# =========================
# Config
# =========================
TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"
MODEL_PATH = "brf_robust_onehot.joblib"
RANDOM_STATE = 42
N_SPLITS = 5

# =========================
# Load
# =========================
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

if TARGET_COL not in train_df.columns:
    raise ValueError(f"'{TARGET_COL}' not in train.csv")

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)

has_test_label = TARGET_COL in test_df.columns
X_test = test_df.drop(columns=[TARGET_COL]) if has_test_label else test_df.copy()
y_test = test_df[TARGET_COL].astype(int) if has_test_label else None

# =========================
# Column typing (자동 추정)
# =========================
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

# =========================
# Model: Balanced Random Forest
# =========================
brf = BalancedRandomForestClassifier(
    n_estimators=800,
    max_depth=None,
    min_samples_leaf=1,
    sampling_strategy="auto",   # 각 트리 학습 시 클래스 균형 샘플링
    replacement=False,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", brf),
])

# =========================
# 1) OOF로 threshold 튜닝 (추가 split 없음)
# =========================
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
oof_proba = np.zeros(len(X_train), dtype=float)

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_train, y_train), 1):
    X_tr, y_tr = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    X_va, y_va = X_train.iloc[va_idx], y_train.iloc[va_idx]

    pipeline.fit(X_tr, y_tr)
    oof_proba[va_idx] = pipeline.predict_proba(X_va)[:, 1]
    print(f"[Fold {fold}] done")

prec, rec, thr = precision_recall_curve(y_train, oof_proba)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = np.nanargmax(f1s)
best_thr = thr[best_idx - 1] if best_idx > 0 else 0.5

print("\n[OOF] Best threshold:", float(best_thr), "Best F1:", float(f1s[best_idx]))

# =========================
# 2) train 전체로 재학습
# =========================
pipeline.fit(X_train, y_train)

# =========================
# 3) test 평가(라벨 있으면)
# =========================
test_proba = pipeline.predict_proba(X_test)[:, 1]
test_pred = (test_proba >= best_thr).astype(int)

if has_test_label:
    print("\n=== TEST Metrics (BRF, OOF-threshold) ===")
    print("ROC-AUC :", roc_auc_score(y_test, test_proba))
    print("PR-AUC  :", average_precision_score(y_test, test_proba))
    print("F1      :", f1_score(y_test, test_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, test_pred))
    print("\nReport:\n", classification_report(y_test, test_pred, digits=4))
else:
    print("\n(test.csv에 Revenue가 없어 평가 대신 예측만 생성합니다.)")

# =========================
# 4) 모델 추출/저장 (pipeline + threshold)
# =========================
artifact = {
    "pipeline": pipeline,
    "best_threshold": float(best_thr),
    "num_cols": num_cols,
    "cat_cols": cat_cols
}
joblib.dump(artifact, MODEL_PATH)
print("\nSaved:", MODEL_PATH)


In [ ]:
# pip install pandas numpy scikit-learn imbalanced-learn joblib

import numpy as np
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_recall_curve, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report
)

from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import EasyEnsembleClassifier

TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"
MODEL_PATH = "easyensemble_robust_onehot.joblib"
RANDOM_STATE = 42
N_SPLITS = 5

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)

has_test_label = TARGET_COL in test_df.columns
X_test = test_df.drop(columns=[TARGET_COL]) if has_test_label else test_df.copy()
y_test = test_df[TARGET_COL].astype(int) if has_test_label else None

num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

# EasyEnsemble은 언더샘플링된 여러 서브셋을 만들고, 각 서브셋에 약한 분류기를 학습한 뒤 앙상블
base = LogisticRegression(max_iter=5000, solver="saga", class_weight=None, n_jobs=-1, random_state=RANDOM_STATE)

eec = EasyEnsembleClassifier(
    n_estimators=30,          # 서브셋 개수(늘리면 더 안정적/느려짐)
    estimator=base,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", eec),
])

# OOF threshold 튜닝
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
oof_proba = np.zeros(len(X_train), dtype=float)

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_train, y_train), 1):
    X_tr, y_tr = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    X_va, y_va = X_train.iloc[va_idx], y_train.iloc[va_idx]

    pipeline.fit(X_tr, y_tr)
    oof_proba[va_idx] = pipeline.predict_proba(X_va)[:, 1]
    print(f"[Fold {fold}] done")

prec, rec, thr = precision_recall_curve(y_train, oof_proba)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = np.nanargmax(f1s)
best_thr = thr[best_idx - 1] if best_idx > 0 else 0.5

print("\n[OOF] Best threshold:", float(best_thr), "Best F1:", float(f1s[best_idx]))

# train 전체 재학습
pipeline.fit(X_train, y_train)

# test 평가
test_proba = pipeline.predict_proba(X_test)[:, 1]
test_pred = (test_proba >= best_thr).astype(int)

if has_test_label:
    print("\n=== TEST Metrics (EasyEnsemble, OOF-threshold) ===")
    print("ROC-AUC :", roc_auc_score(y_test, test_proba))
    print("PR-AUC  :", average_precision_score(y_test, test_proba))
    print("F1      :", f1_score(y_test, test_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, test_pred))
    print("\nReport:\n", classification_report(y_test, test_pred, digits=4))

artifact = {"pipeline": pipeline, "best_threshold": float(best_thr), "num_cols": num_cols, "cat_cols": cat_cols}
joblib.dump(artifact, MODEL_PATH)
print("\nSaved:", MODEL_PATH)


In [ ]:
import pandas as pd

train_df = pd.read_csv("../../data/processed/train.csv")
test_df  = pd.read_csv("../../data/processed/test.csv")

for name, df in [("train", train_df), ("test", test_df)]:
    if "Revenue" in df.columns:
        vc = df["Revenue"].astype(int).value_counts().sort_index()
        pos = int(vc.get(1, 0)); neg = int(vc.get(0, 0))
        print(f"{name}: pos={pos}, neg={neg}, pos_rate={pos/(pos+neg):.4f}")


In [ ]:
# pip install pandas numpy scikit-learn imbalanced-learn joblib

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import (
    precision_recall_curve, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report
)

from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"
RANDOM_STATE = 42
N_SPLITS = 5

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)

X_test = test_df.drop(columns=[TARGET_COL])
y_test = test_df[TARGET_COL].astype(int)

# 자동 컬럼 타입
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

def oof_best_threshold(model_pipeline, X, y, n_splits=5, seed=42):
    """train만으로 OOF 확률 생성 -> F1 최대 threshold 반환"""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(X), dtype=float)

    for tr_idx, va_idx in skf.split(X, y):
        model_pipeline.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        oof[va_idx] = model_pipeline.predict_proba(X.iloc[va_idx])[:, 1]

    prec, rec, thr = precision_recall_curve(y, oof)
    f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
    best_idx = np.nanargmax(f1s)
    best_thr = thr[best_idx - 1] if best_idx > 0 else 0.5
    return float(best_thr), float(f1s[best_idx])

def fit_eval(name, base_model):
    pipe = Pipeline([("preprocess", preprocess), ("model", base_model)])

    best_thr, best_oof_f1 = oof_best_threshold(pipe, X_train, y_train, n_splits=N_SPLITS, seed=RANDOM_STATE)

    # train 전체로 재학습
    pipe.fit(X_train, y_train)

    test_proba = pipe.predict_proba(X_test)[:, 1]
    test_pred = (test_proba >= best_thr).astype(int)

    out = {
        "name": name,
        "oof_best_f1": best_oof_f1,
        "best_thr": best_thr,
        "test_f1": float(f1_score(y_test, test_pred)),
        "test_roc_auc": float(roc_auc_score(y_test, test_proba)),
        "test_pr_auc": float(average_precision_score(y_test, test_proba)),
        "pipe": pipe
    }
    return out, test_pred, test_proba

models = [
    ("LR_balanced", LogisticRegression(max_iter=5000, solver="saga", class_weight="balanced", n_jobs=-1, random_state=RANDOM_STATE)),
    ("BalancedRF", BalancedRandomForestClassifier(n_estimators=800, n_jobs=-1, random_state=RANDOM_STATE)),
    ("EasyEnsemble", EasyEnsembleClassifier(
        n_estimators=30,
        estimator=LogisticRegression(max_iter=5000, solver="saga", n_jobs=-1, random_state=RANDOM_STATE),
        n_jobs=-1,
        random_state=RANDOM_STATE
    )),
    ("ExtraTrees_balanced", ExtraTreesClassifier(n_estimators=800, class_weight="balanced", n_jobs=-1, random_state=RANDOM_STATE)),
]

results = []
best = None

for name, m in models:
    res, test_pred, test_proba = fit_eval(name, m)
    results.append(res)
    print(f"\n[{name}] OOF_best_F1={res['oof_best_f1']:.4f}, best_thr={res['best_thr']:.4f} | "
          f"TEST F1={res['test_f1']:.4f}, PR-AUC={res['test_pr_auc']:.4f}, ROC-AUC={res['test_roc_auc']:.4f}")

    if best is None or res["test_f1"] > best["test_f1"]:
        best = res

print("\n=== BEST by TEST F1 ===")
print(best["name"], "TEST F1=", best["test_f1"], "best_thr=", best["best_thr"])

# 최종 상세 리포트(베스트 모델로)
best_pipe = best["pipe"]
best_thr  = best["best_thr"]
best_test_proba = best_pipe.predict_proba(X_test)[:, 1]
best_test_pred  = (best_test_proba >= best_thr).astype(int)

print("\nConfusion matrix:\n", confusion_matrix(y_test, best_test_pred))
print("\nReport:\n", classification_report(y_test, best_test_pred, digits=4))


In [ ]:
# pip install pandas numpy scikit-learn imbalanced-learn joblib

import numpy as np
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_recall_curve, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report
)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler

TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"
RANDOM_STATE = 42
N_SPLITS = 5

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)

X_test = test_df.drop(columns=[TARGET_COL])
y_test = test_df[TARGET_COL].astype(int)

# ---- 컬럼 자동 분리 (원하면 여기서 수동 지정 가능)
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

def oof_best_threshold(pipeline, X, y, n_splits=5, seed=42):
    """train에서만 OOF 확률 생성 -> F1 최대 threshold 반환"""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(X), dtype=float)

    for tr_idx, va_idx in skf.split(X, y):
        pipeline.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        oof[va_idx] = pipeline.predict_proba(X.iloc[va_idx])[:, 1]

    prec, rec, thr = precision_recall_curve(y, oof)
    f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
    best_idx = int(np.nanargmax(f1s))
    best_thr = float(thr[best_idx - 1]) if best_idx > 0 else 0.5
    return best_thr, float(f1s[best_idx])

def fit_eval(name, pipe):
    # 1) OOF로 best threshold
    thr, oof_f1 = oof_best_threshold(pipe, X_train, y_train, n_splits=N_SPLITS, seed=RANDOM_STATE)

    # 2) train 전체로 재학습
    pipe.fit(X_train, y_train)

    # 3) test 평가
    proba = pipe.predict_proba(X_test)[:, 1]
    pred = (proba >= thr).astype(int)

    return {
        "name": name,
        "oof_best_f1": oof_f1,
        "best_thr": thr,
        "test_f1": float(f1_score(y_test, pred)),
        "test_pr_auc": float(average_precision_score(y_test, proba)),
        "test_roc_auc": float(roc_auc_score(y_test, proba)),
        "pipe": pipe,
        "test_pred": pred,
        "test_proba": proba
    }

# ---- 후보 모델들 (F1 잘 나오는 쪽 위주)
candidates = []

# 1) ExtraTrees (강추: tabular에서 F1 잘 나오기 쉬움)
et = ExtraTreesClassifier(
    n_estimators=1200,
    min_samples_leaf=2,
    class_weight="balanced",
    n_jobs=-1,
    random_state=RANDOM_STATE
)
candidates.append(("ExtraTrees_balanced",
                   Pipeline([("preprocess", preprocess), ("model", et)])))

# 2) Balanced Random Forest
brf = BalancedRandomForestClassifier(
    n_estimators=900,
    min_samples_leaf=1,
    n_jobs=-1,
    random_state=RANDOM_STATE
)
candidates.append(("BalancedRF",
                   Pipeline([("preprocess", preprocess), ("model", brf)])))

# 3) EasyEnsemble (양성이 적을 때 특히 안정적인 경우 많음)
base_lr = LogisticRegression(max_iter=5000, solver="saga", n_jobs=-1, random_state=RANDOM_STATE)
eec = EasyEnsembleClassifier(
    n_estimators=40,
    estimator=base_lr,
    n_jobs=-1,
    random_state=RANDOM_STATE
)
candidates.append(("EasyEnsemble",
                   Pipeline([("preprocess", preprocess), ("model", eec)])))

# 4) (옵션) RandomOverSampler + LR (가중치 대신 표본 보강)
ros_lr = LogisticRegression(max_iter=5000, solver="saga", n_jobs=-1, random_state=RANDOM_STATE)
candidates.append(("ROS+LR",
                   ImbPipeline([("preprocess", preprocess),
                               ("ros", RandomOverSampler(random_state=RANDOM_STATE)),
                               ("model", ros_lr)])))

# ---- 실행
results = []
best = None

for name, pipe in candidates:
    res = fit_eval(name, pipe)
    results.append(res)
    print(f"\n[{name}] OOF_best_F1={res['oof_best_f1']:.4f}, thr={res['best_thr']:.4f} | "
          f"TEST F1={res['test_f1']:.4f}, PR-AUC={res['test_pr_auc']:.4f}, ROC-AUC={res['test_roc_auc']:.4f}")

    if best is None or res["test_f1"] > best["test_f1"]:
        best = res

print("\n=== BEST (by TEST F1) ===")
print(best["name"], "TEST F1=", best["test_f1"], "thr=", best["best_thr"])

print("\nConfusion matrix:\n", confusion_matrix(y_test, best["test_pred"]))
print("\nReport:\n", classification_report(y_test, best["test_pred"], digits=4))

# ---- 저장(모델 추출)
artifact = {
    "pipeline": best["pipe"],
    "best_threshold": float(best["best_thr"]),
    "model_name": best["name"],
    "num_cols": num_cols,
    "cat_cols": cat_cols
}
joblib.dump(artifact, "best_f1_model.joblib")
print("\nSaved: best_f1_model.joblib")


/opt/miniconda3/envs/ai_basic_env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/envs/ai_basic_env/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# pip install pandas numpy scikit-learn imbalanced-learn joblib

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score, average_precision_score

from imblearn.ensemble import BalancedRandomForestClassifier

TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"
RANDOM_STATE = 42
N_SPLITS = 5

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)
X_test  = test_df.drop(columns=[TARGET_COL])
y_test  = test_df[TARGET_COL].astype(int)

# 자동 컬럼 타입
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

def oof_proba_for_pipe(pipe, X, y, n_splits=5, seed=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(X), dtype=float)
    for tr_idx, va_idx in skf.split(X, y):
        pipe.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        oof[va_idx] = pipe.predict_proba(X.iloc[va_idx])[:, 1]
    return oof

def best_f1_threshold(y_true, proba):
    prec, rec, thr = precision_recall_curve(y_true, proba)
    f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
    best_idx = int(np.nanargmax(f1s))
    best_thr = float(thr[best_idx - 1]) if best_idx > 0 else 0.5
    return best_thr, float(f1s[best_idx])

# ---- 튜닝 그리드 (너무 크면 오래 걸리니 "F1 잘 움직이는 축"만 잡음)
grid = []
for max_depth in [None, 8, 12, 16]:
    for min_leaf in [1, 2, 4, 8]:
        for max_features in ["sqrt", 0.5, 0.8]:
            grid.append({
                "n_estimators": 1200,
                "max_depth": max_depth,
                "min_samples_leaf": min_leaf,
                "min_samples_split": 2,
                "max_features": max_features,
                "sampling_strategy": "auto",
                "replacement": False,
            })

best = None

for i, params in enumerate(grid, 1):
    brf = BalancedRandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        min_samples_leaf=params["min_samples_leaf"],
        min_samples_split=params["min_samples_split"],
        max_features=params["max_features"],
        sampling_strategy=params["sampling_strategy"],
        replacement=params["replacement"],
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    pipe = Pipeline([("preprocess", preprocess), ("model", brf)])

    oof = oof_proba_for_pipe(pipe, X_train, y_train, n_splits=N_SPLITS, seed=RANDOM_STATE)
    thr, oof_f1 = best_f1_threshold(y_train, oof)

    # train 전체 재학습 후 test 평가
    pipe.fit(X_train, y_train)
    test_proba = pipe.predict_proba(X_test)[:, 1]
    test_pred = (test_proba >= thr).astype(int)
    test_f1 = float(f1_score(y_test, test_pred))

    if best is None or test_f1 > best["test_f1"]:
        best = {
            "params": params,
            "thr": thr,
            "oof_f1": oof_f1,
            "test_f1": test_f1,
            "test_pr_auc": float(average_precision_score(y_test, test_proba)),
            "test_roc_auc": float(roc_auc_score(y_test, test_proba)),
        }

    print(f"[{i}/{len(grid)}] depth={params['max_depth']}, leaf={params['min_samples_leaf']}, "
          f"max_feat={params['max_features']} | OOF_F1={oof_f1:.4f} thr={thr:.3f} | TEST_F1={test_f1:.4f}")

print("\n=== BEST CONFIG (by TEST F1) ===")
print(best)


In [ ]:
# pip install pandas numpy scikit-learn imbalanced-learn joblib

import numpy as np
import pandas as pd
import joblib

from pathlib import Path

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import average_precision_score, roc_auc_score

from imblearn.ensemble import BalancedRandomForestClassifier

TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"
OUT_PATH   = "artifacts/best_pr_auc_balancedrf.joblib"
RANDOM_STATE = 42

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)
X_test  = test_df.drop(columns=[TARGET_COL])
y_test  = test_df[TARGET_COL].astype(int)

num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

base = BalancedRandomForestClassifier(
    n_jobs=-1,
    random_state=RANDOM_STATE
)

pipe = Pipeline([
    ("preprocess", preprocess),
    ("model", base),
])

# PR-AUC 최적화용 탐색 공간 (너무 크게 잡지 말고, 영향 큰 축 위주)
param_dist = {
    "model__n_estimators": [300, 500, 800, 1200],
    "model__max_depth": [None, 8, 12, 16, 20],
    "model__min_samples_leaf": [1, 2, 4, 8, 12],
    "model__min_samples_split": [2, 5, 10],
    "model__max_features": ["sqrt", 0.3, 0.5, 0.8],
    "model__sampling_strategy": ["auto", 0.5, 0.7, 1.0],
    "model__replacement": [False, True],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=40,  # 시간 여유 있으면 80~120까지 올리면 더 좋아짐
    scoring="average_precision",  # PR-AUC
    cv=cv,
    refit=True,  # best 모델로 train 전체 재학습
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=2,
)

search.fit(X_train, y_train)

best_model = search.best_estimator_
print("\nBest CV PR-AUC:", search.best_score_)
print("Best params:", search.best_params_)

# test 평가(참고용)
test_proba = best_model.predict_proba(X_test)[:, 1]
print("\nTEST PR-AUC :", average_precision_score(y_test, test_proba))
print("TEST ROC-AUC:", roc_auc_score(y_test, test_proba))

# 저장 (압축 권장)
artifact = {
    "pipeline": best_model,
    "best_params": search.best_params_,
    "cv_best_pr_auc": float(search.best_score_),
    "num_cols": num_cols,
    "cat_cols": cat_cols,
    "target_col": TARGET_COL,
}
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)
joblib.dump(artifact, OUT_PATH, compress=("xz", 3))
print("\nSaved:", OUT_PATH)


In [ ]:
# =========================
# 📊 시각화용 공통 설정
# =========================
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.1)

# RandomizedSearchCV 결과를 DataFrame으로 변환
cv_results = pd.DataFrame(search.cv_results_)
cv_results["mean_pr_auc"] = cv_results["mean_test_score"]  # 이름을 더 직관적으로


# =========================
# 1. CV vs Test 성능 요약 막대그래프
# =========================

cv_pr_auc = float(search.best_score_)                 # = Best CV PR-AUC
test_pr_auc = float(average_precision_score(y_test, test_proba))
test_roc_auc = float(roc_auc_score(y_test, test_proba))

metrics_df = pd.DataFrame(
    {
        "metric": ["CV PR-AUC", "Test PR-AUC", "Test ROC-AUC"],
        "value": [cv_pr_auc, test_pr_auc, test_roc_auc],
    }
)

plt.figure(figsize=(6, 4))
ax = sns.barplot(data=metrics_df, x="metric", y="value", palette="Blues_d", legend=False, hue='metric')

for container in ax.containers:
    ax.bar_label(container, fmt="%.3f", padding=3)
plt.ylim(0, 1.0)
# "Best PR-AUC 모델 성능 요약 (CV vs Test)
plt.title("Best PR-AUC Model Performance Summary (CV vs. Test)")
plt.ylabel("Score")
plt.xlabel("")
plt.tight_layout()
plt.show()
# -> PPT에 "성능 요약" 슬라이드로 바로 캡처해서 사용


# =========================
# 2. 하이퍼파라미터별 PR-AUC 영향 시각화
#    (각 파라미터 vs mean PR-AUC)
# =========================

params_to_plot = [
    "param_model__n_estimators",
    "param_model__max_depth",
    "param_model__min_samples_leaf",
    "param_model__min_samples_split",
    "param_model__max_features",
    "param_model__sampling_strategy",
    "param_model__replacement",
]

def plot_param_effects(cv_results, params, score_col="mean_pr_auc"):
    n = len(params)
    ncols = 2
    nrows = int(np.ceil(n / ncols))

    fig, axes = plt.subplots(nrows, ncols, figsize=(12, 4 * nrows))
    axes = axes.ravel()

    for ax, param in zip(axes, params):
        if param not in cv_results.columns:
            ax.set_visible(False)
            continue

        # 값 목록
        values = cv_results[param].dropna().unique()

        # 숫자인지/문자인지 구분
        def is_numeric(v):
            return isinstance(v, (int, float, np.number))

        if all(is_numeric(v) for v in values):
            # 숫자형 파라미터: 값별 평균 PR-AUC를 선그래프로
            tmp = (
                cv_results.groupby(param)[score_col]
                .mean()
                .reset_index()
                .sort_values(param)
            )
            sns.lineplot(data=tmp, x=param, y=score_col, marker="o", ax=ax)
        else:
            # 범주형 파라미터: 박스플롯/포인트플롯
            tmp = cv_results[[param, score_col]].copy()
            tmp[param] = tmp[param].astype(str)
            sns.boxplot(data=tmp, x=param, y=score_col, ax=ax)
            ax.tick_params("x", labelrotation=30)

        ax.set_title(param.replace("param_model__", "model__"))
        ax.set_ylabel("mean CV PR-AUC")
        ax.set_xlabel("")

    # 남는 subplot 숨기기
    for i in range(len(params), len(axes)):
        axes[i].set_visible(False)

    # Hyperparameter 별 PR-AUC 수치 변화
    plt.suptitle("Changes in CV PR-AUC by hyperparameter", y=1.02, fontsize=14)
    plt.tight_layout()
    plt.show()


plot_param_effects(cv_results, params_to_plot)


# =========================
# 3. Best vs 다른 후보들 비교 (Ranking Plot)
# =========================

# 상위 k개만 보기 (노이즈 줄이기용)
k = 20
topk = (
    cv_results.sort_values("mean_pr_auc", ascending=False)
    .head(k)
    .reset_index(drop=True)
)

plt.figure(figsize=(6, 5))
sns.lineplot(
    data=topk,
    x=topk.index + 1,
    y="mean_pr_auc",
    marker="o",
    sort=False,
)

# 무작위 탐색 상위 {k}개 구성의 CV PR-AUC
plt.title(f"CV PR-AUC of top {k} configurations from random search")
plt.xlabel("Rank (1 = best)")
plt.ylabel("mean CV PR-AUC")
plt.xticks(range(1, k + 1))
plt.tight_layout()
plt.show()


In [ ]:
# ---- 튜닝 그리드 (너무 크면 오래 걸리니 "F1 잘 움직이는 축"만 잡음)
grid = []
for max_depth in [None, 8, 12, 16]:
    for min_leaf in [1, 2, 4, 8]:
        for max_features in ["sqrt", 0.5, 0.8]:
            grid.append({
                "n_estimators": 1200,
                "max_depth": max_depth,
                "min_samples_leaf": min_leaf,
                "min_samples_split": 2,
                "max_features": max_features,
                "sampling_strategy": "auto",
                "replacement": False,
            })

best = None
results = []   # 👈 모든 조합의 결과를 담을 리스트 추가

for i, params in enumerate(grid, 1):
    brf = BalancedRandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        min_samples_leaf=params["min_samples_leaf"],
        min_samples_split=params["min_samples_split"],
        max_features=params["max_features"],
        sampling_strategy=params["sampling_strategy"],
        replacement=params["replacement"],
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    pipe = Pipeline([("preprocess", preprocess), ("model", brf)])

    oof = oof_proba_for_pipe(pipe, X_train, y_train, n_splits=N_SPLITS, seed=RANDOM_STATE)
    thr, oof_f1 = best_f1_threshold(y_train, oof)

    # train 전체 재학습 후 test 평가
    pipe.fit(X_train, y_train)
    test_proba = pipe.predict_proba(X_test)[:, 1]
    test_pred = (test_proba >= thr).astype(int)
    test_f1 = float(f1_score(y_test, test_pred))
    test_pr_auc = float(average_precision_score(y_test, test_proba))
    test_roc_auc = float(roc_auc_score(y_test, test_proba))

    # 👇 여기서 각 조합의 결과를 results에 저장
    results.append({
        **params,                # 각 하이퍼파라미터 펼쳐서 저장
        "thr": thr,
        "oof_f1": oof_f1,
        "test_f1": test_f1,
        "test_pr_auc": test_pr_auc,
        "test_roc_auc": test_roc_auc,
    })

    if best is None or test_f1 > best["test_f1"]:
        best = {
            "params": params,
            "thr": thr,
            "oof_f1": oof_f1,
            "test_f1": test_f1,
            "test_pr_auc": test_pr_auc,
            "test_roc_auc": test_roc_auc,
        }

    print(f"[{i}/{len(grid)}] depth={params['max_depth']}, leaf={params['min_samples_leaf']}, "
          f"max_feat={params['max_features']} | OOF_F1={oof_f1:.4f} thr={thr:.3f} | TEST_F1={test_f1:.4f}")

print("\n=== BEST CONFIG (by TEST F1) ===")
print(best)


In [6]:
# =====================================
# 📊 시각화 (F1 튜닝 결과용)
# =====================================
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.1)

# 1) results를 DataFrame으로 변환
results_df = pd.DataFrame(results)

print("\n[INFO] 총 시도한 조합 수:", len(results_df))

results = []
# -----------------------------
# 1. Best config 성능 요약 막대그래프
# -----------------------------
best_thr = best["thr"]
best_test_f1 = best["test_f1"]
best_test_pr_auc = best["test_pr_auc"]
best_test_roc_auc = best["test_roc_auc"]

metrics_df = pd.DataFrame(
    {
        "metric": ["Best OOF F1", "Best Test F1", "Test PR-AUC", "Test ROC-AUC"],
        "value": [best["oof_f1"], best_test_f1, best_test_pr_auc, best_test_roc_auc],
    }
)

plt.figure(figsize=(7, 4))
ax = sns.barplot(data=metrics_df, x="metric", y="value", palette="viridis", legend=False, hue='metric')

for container in ax.containers:
    ax.bar_label(container, fmt="%.3f", padding=3)
plt.ylim(0, 1.0)
# Best F1 모델 성능 요약 (OOF vs Test)
plt.title("Best F1 Model Performance Summary (OOF vs. Test)")
plt.ylabel("Score")
plt.xlabel("")
plt.tight_layout()
plt.show()


# -----------------------------
# 2. 주요 하이퍼파라미터 vs Test F1
#    (max_depth, min_samples_leaf, max_features)
# -----------------------------
def oof_proba_for_pipe(pipe, X, y, n_splits=5, seed=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(X), dtype=float)
    for tr_idx, va_idx in skf.split(X, y):
        pipe.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        oof[va_idx] = pipe.predict_proba(X.iloc[va_idx])[:, 1]
    return oof


def plot_param_effects_f1(df, score_col="test_f1"):
    params_to_plot = ["max_depth", "min_samples_leaf", "max_features"]
    n = len(params_to_plot)
    ncols = 3
    nrows = 1

    fig, axes = plt.subplots(nrows, ncols, figsize=(16, 4))
    axes = axes.ravel()

    for ax, param in zip(axes, params_to_plot):
        if param not in df.columns:
            ax.set_visible(False)
            continue

        values = df[param].unique()

        # 숫자형인지 체크
        def is_numeric(v):
            return isinstance(v, (int, float, np.number))

        if all(is_numeric(v) for v in values):
            tmp = (
                df.groupby(param)[score_col]
                .mean()
                .reset_index()
                .sort_values(param)
            )
            sns.lineplot(data=tmp, x=param, y=score_col, marker="o", ax=ax)
        else:
            # None이 섞여 있거나 문자열이 있을 때: 카테고리로 처리
            tmp = df[[param, score_col]].copy()
            tmp[param] = tmp[param].astype(str)
            sns.boxplot(data=tmp, x=param, y=score_col, ax=ax)
            ax.tick_params("x", labelrotation=30)
            # ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
        
        ax.set_title(param)
        ax.set_ylabel(score_col)
        ax.set_xlabel("")
    # 하이퍼파라미터별 Test F1 평균
    plt.suptitle("Test F1 average by hyperparameter", y=1.05, fontsize=14)
    plt.tight_layout()
    plt.show()


plot_param_effects_f1(results_df)



# -----------------------------
# 3. 상위 k개 조합의 Test F1 랭킹
# -----------------------------
k = 20
topk = (
    results_df.sort_values("test_f1", ascending=False)
    .head(k)
    .reset_index(drop=True)
)

plt.figure(figsize=(7, 4))
sns.lineplot(
    x=topk.index + 1,
    y="test_f1",
    data=topk,
    marker="o",
)
# 랜덤 그리드 상위 {k}개 조합의 Test F1
plt.title(f"Test F1 of the top {k} combinations of random grids")
plt.xlabel("Rank (1 = best)")
plt.ylabel("Test F1")
plt.xticks(range(1, k + 1))
plt.tight_layout()
plt.show()

# [TOP-K PARAMS] 상위 F1 조합 미리보기
print("\n[TOP-K PARAMS] Preview of the top F1 combinations:")
display_cols = ["test_f1", "test_pr_auc", "test_roc_auc", "max_depth", "min_samples_leaf", "max_features"]
print(topk[display_cols].head(10))


# -----------------------------
# 4. 최종 best 모델의 Threshold vs F1 곡선
# -----------------------------
# best params로 다시 모델 학습해 threshold 곡선을 그림
best_params = best["params"]
best_brf = BalancedRandomForestClassifier(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    min_samples_leaf=best_params["min_samples_leaf"],
    min_samples_split=best_params["min_samples_split"],
    max_features=best_params["max_features"],
    sampling_strategy=best_params["sampling_strategy"],
    replacement=best_params["replacement"],
    n_jobs=-1,
    random_state=RANDOM_STATE
)
best_pipe = Pipeline([("preprocess", preprocess), ("model", best_brf)])
best_pipe.fit(X_train, y_train)

best_test_proba = best_pipe.predict_proba(X_test)[:, 1]
prec, rec, thr = precision_recall_curve(y_test, best_test_proba)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)

# precision_recall_curve의 thr 길이는 n-1 이라 x축 정렬을 살짝 맞춰줌
thr_for_plot = np.r_[0.0, thr]  # 앞에 dummy 하나 붙여서 길이 맞추기

plt.figure(figsize=(7, 4))
plt.plot(thr_for_plot, f1s, marker=".")
plt.axvline(best_thr, color="red", linestyle="--", label=f"best thr = {best_thr:.3f}")

# Threshold에 따른 F1 변화 (best F1 모델)
plt.title("F1 change according to threshold (best F1 model)")
plt.xlabel("Threshold")
plt.ylabel("F1 score")
plt.legend()
plt.tight_layout()
plt.show()

# F2 Score 추가 출력
from sklearn.metrics import fbeta_score
best_test_pred = (best_test_proba >= best_thr).astype(int)
test_f2 = fbeta_score(y_test, best_test_pred, beta=2)
print(f"\n[Test Metrics] F2-Score (beta=2, thr={best_thr:.3f}): {test_f2:.4f}")


NameError: name 'results' is not defined

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve

sns.set(style="whitegrid", font_scale=1.1)

# 1) 최종 best 설정
best_params = best["params"]
train_best_thr = best["thr"]   # OOF 기준으로 찾은 공식 threshold

best_brf = BalancedRandomForestClassifier(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    min_samples_leaf=best_params["min_samples_leaf"],
    min_samples_split=best_params["min_samples_split"],
    max_features=best_params["max_features"],
    sampling_strategy=best_params["sampling_strategy"],
    replacement=best_params["replacement"],
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

best_pipe = Pipeline([
    ("preprocess", preprocess),
    ("model", best_brf),
])

best_pipe.fit(X_train, y_train)
proba_test = best_pipe.predict_proba(X_test)[:, 1]

prec, rec, thr = precision_recall_curve(y_test, proba_test)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)

# threshold 배열 길이 맞추기
thr_for_plot = np.r_[0.0, thr]

# (1) train에서 찾은 threshold(0.741...)에 가장 가까운 F1 값 찾기
idx_train_thr = int(np.argmin(np.abs(thr_for_plot - train_best_thr)))
f1_at_train_thr = f1s[idx_train_thr]

# (2) test 기준 F1 최고 threshold도 계산해 보고 싶다면 (선택)
idx_test_best = int(np.nanargmax(f1s))
test_best_thr = float(thr_for_plot[idx_test_best])
test_best_f1 = float(f1s[idx_test_best])

plt.figure(figsize=(7, 4))
plt.plot(thr_for_plot, f1s, marker=".", label="F1 score")

# 붉은 선: train/OOF 기준 best threshold
plt.axvline(train_best_thr, color="red", linestyle="--",
            label=f"train best thr = {train_best_thr:.3f}")
plt.scatter([train_best_thr], [f1_at_train_thr],
            color="red", s=60, zorder=5)

# 회색 점선 (선택): test 기준 peak
plt.axvline(test_best_thr, color="gray", linestyle=":",
            label=f"test peak thr = {test_best_thr:.3f}")
plt.scatter([test_best_thr], [test_best_f1],
            color="gray", s=50, zorder=5)

plt.title("F1 Score Changes According to Threshold (Best F1 Model)")
plt.xlabel("Threshold")
plt.ylabel("F1 score")
plt.legend(loc="best")
plt.tight_layout()
# plt.savefig("fig_5_4_threshold_vs_f1.png", dpi=150, bbox_inches="tight")
plt.show()


In [7]:
# =====================================
# 📊 F2 Score Optimization & Visualization (Standalone Script)
# =====================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, f1_score, average_precision_score, roc_auc_score, fbeta_score

sns.set(style="whitegrid", font_scale=1.1)

# -----------------------------
# 1. Configuration & Data Loading
# -----------------------------
TRAIN_PATH = "../../data/processed/train.csv"
TEST_PATH  = "../../data/processed/test.csv"
TARGET_COL = "Revenue"
RANDOM_STATE = 42
N_SPLITS = 5

print("Loading Data...")
try:
    train_df = pd.read_csv(TRAIN_PATH)
    test_df  = pd.read_csv(TEST_PATH)
except FileNotFoundError:
    # Fallback to absolute path or adjust as necessary if running elsewhere
    import os
    print(f"Data not found at {TRAIN_PATH}. Checking current directory...")
    # Assuming script is run from notebooks/EomHyungEun/
    base_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
    # Adjust logic to find data if needed
    raise FileNotFoundError("Please ensure train.csv and test.csv are in ../../data/processed/")

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL].astype(int)
X_test  = test_df.drop(columns=[TARGET_COL])
y_test  = test_df[TARGET_COL].astype(int)

# -----------------------------
# 2. Preprocessing
# -----------------------------
print("Setting up Preprocessing...")
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool", "category"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

# -----------------------------
# 3. Helper Functions
# -----------------------------
def oof_proba_for_pipe(pipe, X, y, n_splits=5, seed=42):
    """
    Perform Out-Of-Fold probability prediction.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(X), dtype=float)
    for tr_idx, va_idx in skf.split(X, y):
        X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
        X_va = X.iloc[va_idx]
        
        pipe.fit(X_tr, y_tr)
        oof[va_idx] = pipe.predict_proba(X_va)[:, 1]
    return oof

def best_f1_threshold(y_true, proba):
    """
    Find the threshold that maximizes F1 score.
    """
    prec, rec, thr = precision_recall_curve(y_true, proba)
    f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
    best_idx = int(np.nanargmax(f1s))
    best_thr = float(thr[best_idx - 1]) if best_idx > 0 else 0.5
    return best_thr, float(f1s[best_idx])

# -----------------------------
# 4. Model Training & Grid Search
# -----------------------------
print("Starting Grid Search...")

# grid definition (Partial grid as per notebook)
grid = []
for max_depth in [None, 8, 12, 16]:
    for min_leaf in [1, 2, 4, 8]:
        for max_features in ["sqrt", 0.5, 0.8]:
            grid.append({
                "n_estimators": 1200,
                "max_depth": max_depth,
                "min_samples_leaf": min_leaf,
                "min_samples_split": 2,
                "max_features": max_features,
                "sampling_strategy": "auto",
                "replacement": False,
            })

best = None
results = []

total_iter = len(grid)
for i, params in enumerate(grid, 1):
    brf = BalancedRandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        min_samples_leaf=params["min_samples_leaf"],
        min_samples_split=params["min_samples_split"],
        max_features=params["max_features"],
        sampling_strategy=params["sampling_strategy"],
        replacement=params["replacement"],
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    pipe = Pipeline([("preprocess", preprocess), ("model", brf)])

    # OOF Prediction for validation Score
    oof = oof_proba_for_pipe(pipe, X_train, y_train, n_splits=N_SPLITS, seed=RANDOM_STATE)
    thr, oof_f1 = best_f1_threshold(y_train, oof)

    # Train on full train set for test eval
    pipe.fit(X_train, y_train)
    test_proba = pipe.predict_proba(X_test)[:, 1]
    
    # Eval on Test
    test_pred = (test_proba >= thr).astype(int)
    test_f1 = float(f1_score(y_test, test_pred))
    
    current_result = {
        **params,
        "thr": thr,
        "oof_f1": oof_f1,
        "test_f1": test_f1,
        "test_pr_auc": float(average_precision_score(y_test, test_proba)),
        "test_roc_auc": float(roc_auc_score(y_test, test_proba)),
    }
    results.append(current_result)
    
    if best is None or test_f1 > best["test_f1"]:
        best = current_result.copy()
        best["params"] = params # Store params separately to match accessing logic later
    
    print(f"[{i}/{total_iter}] depth={params['max_depth']}, leaf={params['min_samples_leaf']}, "
          f"max_feat={params['max_features']} | OOF_F1={oof_f1:.4f} thr={thr:.3f} | TEST_F1={test_f1:.4f}")

print("\n=== BEST CONFIG (by TEST F1) ===")
print(best)

# -----------------------------
# 5. Visualization & F2 Score
# -----------------------------
print("\nGenerating Visualizations...")

results_df = pd.DataFrame(results)
print("\n[INFO] Total combinations trained:", len(results_df))

# --- Plot 1: Best config performance ---
best_thr = best["thr"]
best_test_f1 = best["test_f1"]
best_test_pr_auc = best["test_pr_auc"]
best_test_roc_auc = best["test_roc_auc"]

metrics_df = pd.DataFrame(
    {
        "metric": ["Best OOF F1", "Best Test F1", "Test PR-AUC", "Test ROC-AUC"],
        "value": [best["oof_f1"], best_test_f1, best_test_pr_auc, best_test_roc_auc],
    }
)

plt.figure(figsize=(7, 4))
ax = sns.barplot(data=metrics_df, x="metric", y="value", palette="viridis", legend=False, hue='metric')
for container in ax.containers:
    ax.bar_label(container, fmt="%.3f", padding=3)
plt.ylim(0, 1.0)
plt.title("Best F1 Model Performance Summary (OOF vs. Test)")
plt.ylabel("Score")
plt.xlabel("")
plt.tight_layout()
plt.show()

# --- Plot 2: Hyperparameter Effects ---
def plot_param_effects_f1(df, score_col="test_f1"):
    params_to_plot = ["max_depth", "min_samples_leaf", "max_features"]
    n = len(params_to_plot)
    ncols = 3
    nrows = 1

    fig, axes = plt.subplots(nrows, ncols, figsize=(16, 4))
    axes = axes.ravel()

    for ax, param in zip(axes, params_to_plot):
        if param not in df.columns:
            ax.set_visible(False)
            continue

        values = df[param].unique()
        def is_numeric(v):
            return isinstance(v, (int, float, np.number))

        if all(is_numeric(v) for v in values):
            tmp = df.groupby(param)[score_col].mean().reset_index().sort_values(param)
            sns.lineplot(data=tmp, x=param, y=score_col, marker="o", ax=ax)
        else:
            tmp = df[[param, score_col]].copy()
            tmp[param] = tmp[param].astype(str)
            sns.boxplot(data=tmp, x=param, y=score_col, ax=ax)
            ax.tick_params("x", labelrotation=30)
        
        ax.set_title(param)
        ax.set_ylabel(score_col)
        ax.set_xlabel("")
    plt.suptitle("Test F1 average by hyperparameter", y=1.05, fontsize=14)
    plt.tight_layout()
    plt.show()

plot_param_effects_f1(results_df)

# --- Plot 3: Top K Ranking ---
k = 20
topk = results_df.sort_values("test_f1", ascending=False).head(k).reset_index(drop=True)

plt.figure(figsize=(7, 4))
sns.lineplot(x=topk.index + 1, y="test_f1", data=topk, marker="o")
plt.title(f"Test F1 of the top {k} combinations of random grids")
plt.xlabel("Rank (1 = best)")
plt.ylabel("Test F1")
plt.xticks(range(1, k + 1))
plt.tight_layout()
plt.show()

print("\n[TOP-K PARAMS] Preview of the top F1 combinations:")
display_cols = ["test_f1", "test_pr_auc", "test_roc_auc", "max_depth", "min_samples_leaf", "max_features"]
print(topk[display_cols].head(10))

# --- Plot 4: Threshold vs F1 Curve for Best Model ---
best_params = best["params"]
best_brf = BalancedRandomForestClassifier(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    min_samples_leaf=best_params["min_samples_leaf"],
    min_samples_split=best_params["min_samples_split"],
    max_features=best_params["max_features"],
    sampling_strategy=best_params["sampling_strategy"],
    replacement=best_params["replacement"],
    n_jobs=-1,
    random_state=RANDOM_STATE
)
best_pipe = Pipeline([("preprocess", preprocess), ("model", best_brf)])
best_pipe.fit(X_train, y_train)

best_test_proba = best_pipe.predict_proba(X_test)[:, 1]
prec, rec, thr = precision_recall_curve(y_test, best_test_proba)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)

thr_for_plot = np.r_[0.0, thr]  # align length

plt.figure(figsize=(7, 4))
plt.plot(thr_for_plot, f1s, marker=".")
plt.axvline(best_thr, color="red", linestyle="--", label=f"best thr = {best_thr:.3f}")
plt.title("F1 change according to threshold (best F1 model)")
plt.xlabel("Threshold")
plt.ylabel("F1 score")
plt.legend()
plt.tight_layout()
plt.show()

# -----------------------------
# 6. F2 Score Calculation
# -----------------------------
best_test_pred = (best_test_proba >= best_thr).astype(int)
test_f2 = fbeta_score(y_test, best_test_pred, beta=2)
print(f"\n[Test Metrics] F2-Score (beta=2, thr={best_thr:.3f}): {test_f2:.4f}")


Loading Data...
Setting up Preprocessing...
Starting Grid Search...
[1/48] depth=None, leaf=1, max_feat=sqrt | OOF_F1=0.6890 thr=0.708 | TEST_F1=0.6667
[2/48] depth=None, leaf=1, max_feat=0.5 | OOF_F1=0.6898 thr=0.738 | TEST_F1=0.6650


KeyboardInterrupt: 